In [1]:
import string
from pathlib import Path
from typing import Optional

import pandas as pd
import rich
from pydantic import BaseModel, TypeAdapter
from tqdm import tqdm

# Import a model to verify the setup
from twfy_vector_explorer import notebook_setup as notebook_setup
from vector_explorer.models import NgramVector

pd.set_option("display.max_colwidth", None)

# Generic search for exploring - ngram

In [5]:
alts = ["mental health"]

false_positives = []


def fix_puntuation_items(t: str) -> str:
    words = t.split(" ")
    # remove any words that are just one bit of punctuation
    other_puntation = "".join(["”", "“", "’", "‘"])
    words = [w for w in words if w not in string.punctuation + other_puntation]
    # remove start or end words that are and, or, etc
    banned_words = [
        "and",
        "or",
        "of",
        "the",
        "in",
        "to",
        "a",
        "on",
        "for",
        "with",
        "by",
        "from",
        "as",
        "at",
        "an",
        "is",
        "are",
        "were",
    ]
    # check for banned words at start and end
    while words and words[0] in banned_words:
        words = words[1:]
    while words and words[-1] in banned_words:
        words = words[:-1]
    return " ".join(words)


def remove_future_contains(series: pd.Series) -> pd.Series:
    """
    Here we're trying to deal with the problem that we
    end up with
    'mental ill health' and 'of mental ill health' 'with mental ill health'
    We want to remove longer versions of the same thing.
    Return a mask that can be used to filter the series.
    We want to prioritise earlier items in the series.
    """
    banned_items = []
    # sort items by length
    sorted_series = series  # .sort_values(key=lambda x: x.str.len())
    for i, item in enumerate(sorted_series):
        remaining_items = sorted_series[i + 1 :]
        for ni, next_item in enumerate(remaining_items):
            if item in next_item:
                banned_items.append(next_item)

    return series.isin(banned_items)


class QueryMatch(BaseModel):
    text: str
    count: int
    distance: float


class SearchQuery(BaseModel):
    query: str
    nearest: list[QueryMatch]


SearchList = TypeAdapter(list[SearchQuery])


def find_related_ngrams(search_query: str) -> Optional[SearchQuery]:
    df = NgramVector.objects.all().search_distance(search_query, threshold=0.15).df()

    if df.empty:
        return None

    df = (
        df.assign(text=lambda df: df["text"].str.strip())
        .drop_duplicates(subset="text", keep="first")
        .drop(columns=["embedding", "id"])
    )

    df = df.sort_values("distance", ascending=True).reset_index(drop=True)

    df["text"] = df["text"].apply(fix_puntuation_items)
    # remove duplicates
    df = df.drop_duplicates(subset="text", keep="first")
    # drop future contains
    future_mask = remove_future_contains(df["text"])
    df = df[~future_mask]

    # remove if text is the original query
    df = df[df["text"] != search_query]

    # no matches that are an exact match - want to find similar items
    response = SearchQuery(
        query=search_query, nearest=df.head(10).to_dict(orient="records")  # type: ignore
    ) 
    return response

In [6]:
items: list[SearchQuery] = []

queries_df = pd.read_csv(Path("..", "data", "sample_queries.csv"))

queries = queries_df["queries"].tolist()

for query in tqdm(queries):
    if not query:
        continue
    result = find_related_ngrams(query)
    if not result:
        continue
    if len(result.nearest) > 0:
        items.append(result)

rich.print(items)

100%|██████████| 997/997 [08:58<00:00,  1.85it/s]


[
    SearchQuery(
        query='keir starmer',
        nearest=[QueryMatch(text='starmer', count=73, distance=0.11694252490997314)]
    ),
    SearchQuery(
        query='enterprise and regulatory reform bill',
        nearest=[
            QueryMatch(text='enterprise and regulatory reform act', count=57, distance=0.06009692670070088),
            QueryMatch(text='regulatory reform bill', count=171, distance=0.06979347107332434),
            QueryMatch(text='enterprise and regulatory reform', count=261, distance=0.09109029988261397),
            QueryMatch(text='regulatory reform public bill', count=16, distance=0.09848997877448251),
            QueryMatch(text='regulatory reform act', count=103, distance=0.12582365053365607),
            QueryMatch(text='legislative and regulatory reform', count=35, distance=0.13293263712323644),
            QueryMatch(text='regulation bill', count=17, distance=0.13305304643985694),
            QueryMatch(text='enterprise and employment bill', count=84, distance=0.13728770088333508),
            QueryMatch(text='regulatory reform', count=93, distance=0.13905852124966955),
            QueryMatch(text='enterprise bill', count=107, distance=0.14351615737425794)
        ]
    ),
    SearchQuery(
        query='competition act',
        nearest=[
            QueryMatch(text='competition law', count=380, distance=0.09271973924158283),
            QueryMatch(text='competition powers', count=11, distance=0.13669666112005463),
            QueryMatch(text='competition rules', count=66, distance=0.1404221141589933),
            QueryMatch(text='competition', count=58, distance=0.14680494628011542)
        ]
    ),
    SearchQuery(
        query='jeremy corbyn',
        nearest=[QueryMatch(text='corbyn', count=581, distance=0.13412559032440186)]
    ),
    SearchQuery(
        query='liz truss',
        nearest=[QueryMatch(text='elizabeth truss', count=196, distance=0.06480846388415629)]
    ),
    SearchQuery(
        query='medical cannabis',
        nearest=[
            QueryMatch(text='cannabis for medical', count=16, distance=0.04481089649330339),
            QueryMatch(text='medicinal cannabis', count=166, distance=0.07445115294761218),
            QueryMatch(text='medical use of cannabis', count=33, distance=0.0861015967191201),
            QueryMatch(text='cannabis for medicinal', count=28, distance=0.11936695095103533),
            QueryMatch(text='cannabis-based medicine', count=19, distance=0.12391865253447865)
        ]
    ),
    SearchQuery(
        query='matt hancock',
        nearest=[QueryMatch(text='matthew hancock', count=171, distance=0.07523230390837488)]
    ),
    SearchQuery(
        query='suella braverman',
        nearest=[QueryMatch(text='braverman', count=11, distance=0.1258649305024686)]
    ),
    SearchQuery(
        query='crown estate',
        nearest=[QueryMatch(text='crown property', count=14, distance=0.07911354774913315)]
    ),
    SearchQuery(
        query='michael gove',
        nearest=[QueryMatch(text='mr gove', count=15, distance=0.13650582461110827)]
    ),
    SearchQuery(
        query='free school meals',
        nearest=[
            QueryMatch(text='free school meal', count=125, distance=0.04024043559311141),
            QueryMatch(text='free school dinners', count=19, distance=0.0581888275798802),
            QueryMatch(text='free school lunches', count=11, distance=0.0627714116207182),
            QueryMatch(text='school meals', count=2055, distance=0.08187034970135498),
            QueryMatch(text='free meals', count=40, distance=0.09350926241333857),
            QueryMatch(text='school meal provision', count=22, distance=0.12527647987832036),
            QueryMatch(text='free school', count=3015, distance=0.12786331391741546),
            QueryMatch(text='school meal', count=178, distance=0.13019582241189664),
            QueryMatch(text='school dinners', count=32, distance=0.1442424740935847)
        ]
    ),
    SearchQuery(query='david lammy'

In [8]:

with Path("..", "data", "search_matches.json").open("wb") as f:
    f.write(SearchList.dump_json(items, indent=2))

In [9]:
rows = []
for item in items:
    for match in item.nearest:
        rows.append(
            {
                "query": item.query,
                "text": match.text, 
                "count": match.count,
                "distance": match.distance
            }
        )

results_df = pd.DataFrame(rows)

results_df.to_csv(Path("..", "data", "search_matches.csv"), index=False)